In [2]:
from googleapiclient.discovery import build
import pandas as pd
import requests
import json
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Get API key from environment variables
API_KEY = os.getenv("YOUTUBE_API_KEY")
CHANNEL_ID = "UCCuzDCoI3EUOo_nhCj4noSw"

# Define date range (March 19–28)
START_DATE = "2025-03-01T00:00:00Z"
END_DATE = "2025-03-30T00:00:00Z"

# Step 1: Get archived live streams
search_url = f"https://www.googleapis.com/youtube/v3/search?part=snippet&channelId={CHANNEL_ID}&type=video&eventType=completed&publishedAfter={START_DATE}&publishedBefore={END_DATE}&maxResults=100&key={API_KEY}"
search_response = requests.get(search_url).json()

video_ids = []
if "items" in search_response:
    for item in search_response["items"]:
        video_id = item["id"]["videoId"]
        video_ids.append(video_id)

# Step 2: Get view counts and other statistics
video_details = []
if video_ids:
    video_ids_str = ",".join(video_ids)  # Join multiple IDs into one request
    stats_url = f"https://www.googleapis.com/youtube/v3/videos?part=snippet,statistics&id={video_ids_str}&key={API_KEY}"
    stats_response = requests.get(stats_url).json()

    # Collect video details
    if "items" in stats_response:
        for item in stats_response["items"]:
            video_info = {
                "title": item["snippet"]["title"],
                "video_url": f"https://www.youtube.com/watch?v={item['id']}",
                "published_date": item["snippet"].get("publishedAt", "N/A"),
                "views": item["statistics"].get("viewCount", "N/A"),
                "likes": item["statistics"].get("likeCount", "N/A"),
                "comments": item["statistics"].get("commentCount", "N/A")
            }
            video_details.append(video_info)

# Print formatted JSON
if video_details:
    print(json.dumps(video_details, indent=4))
else:
    print("No archived live streams found within the given date range.")


[
    {
        "title": "485 17:25 - 21:09 ; 1088 19:45 - 21:42",
        "video_url": "https://www.youtube.com/watch?v=b0yCYRw6nIw",
        "published_date": "2025-03-29T15:52:42Z",
        "views": "4753305",
        "likes": "134279",
        "comments": "3712"
    },
    {
        "title": "MARAPTHON SEASON 2 - Welcome To Thousand Sunny #MARAPTHON #YB #Tierison #LYuka #GarryAng DAY 60 pt.2",
        "video_url": "https://www.youtube.com/watch?v=GvZcgJ98mAk",
        "published_date": "2025-03-21T17:13:33Z",
        "views": "3677961",
        "likes": "37233",
        "comments": "424"
    },
    {
        "title": "MARAPTHON SEASON 2 - Welcome To Thousand Sunny #MARAPTHON #YB #Tierison #LYuka #GarryAng DAY 46 pt.2",
        "video_url": "https://www.youtube.com/watch?v=UhxCmRtHeW4",
        "published_date": "2025-03-07T22:20:05Z",
        "views": "3398756",
        "likes": "41825",
        "comments": "225"
    },
    {
        "title": "MARAPTHON SEASON 2 - Welcome To Thousa